In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


In [2]:
def xaver_init(n_inputs, n_outputs, uniform = True):
    if uniform:
        init_range = tf.sqrt(6.0/ (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)

    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)


In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 50
learning_rate = 0.001


In [4]:

# tf.reset_default_graph()          #clears the memory of all variables left from previous runs(reset the computational graph)


inputs = tf.placeholder(tf.float32, [None,input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

dropout_rate = tf.placeholder("float")


weights_1 = tf.get_variable("weights_1",[input_size, hidden_layer_size],initializer=xaver_init(input_size, hidden_layer_size))
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])

outputs_1 = tf.nn.relu(tf.matmul(inputs,weights_1)+biases_1)

weights_2 = tf.get_variable("weights_2",[hidden_layer_size, output_size],initializer=xaver_init(hidden_layer_size,output_size))
biases_2 = tf.get_variable("biases_2",[output_size])


# weights_3 = tf.get_variable("weights_3",[input_size, hidden_layer_size],initializer=xaver_init(input_size, hidden_layer_size))
# biases_3 = tf.get_variable("biases_3",[hidden_layer_size])

# outputs = tf.matmul(outputs_2,weights_3)+biases_3
_L1 = tf.nn.relu(tf.add(tf.matmul(inputs,weights_1),biases_1))
L1 = tf.nn.dropout(_L1, dropout_rate)
_L2 = tf.nn.relu(tf.add(tf.matmul(L1, weights_2),biases_2)) # Hidden layer with ReLU activation
L2 = tf.nn.dropout(_L2, dropout_rate)

outputs = tf.matmul(outputs_1,weights_2)+biases_2

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:


loss = tf.nn.softmax_cross_entropy_with_logits (logits = outputs, labels = targets)

mean_loss = tf.reduce_mean(loss)

optimize = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)


#Calulate the average of training accuracy of the model

out_equals_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32)) #method that can an object to another data type

with tf.Session() as sess:

    initializer = tf.global_variables_initializer()

    sess.run(initializer)


    batch_size = 100

    max_epochs = 15

    prev_validation_loss = 99999999.


    for epoch_counter in range(max_epochs):
        curr_epoch_loss = 0.

        batches_number = int(mnist.train.num_examples/batch_size)

        for batch_counter in range(batches_number):

            input_batch, target_batch = mnist.train.next_batch(batch_size)   #loads the batches one after the other 

            _, batch_loss = sess.run([ optimize,mean_loss],feed_dict={inputs: input_batch, targets: target_batch, dropout_rate: 0.7})

            curr_epoch_loss += batch_loss

        curr_epoch_loss /= batches_number

        input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)

        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],feed_dict={inputs: input_batch, targets: target_batch, dropout_rate: 0.7})

        print('Epoch '+str(epoch_counter+1)+
              '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
              '. Validation loss: '+'{0:3f}'.format(validation_loss)+
              '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')

        if validation_loss>prev_validation_loss :
            break

        prev_validation_loss = validation_loss


    print ('End of training.')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 1. Training loss: 0.434. Validation loss: 0.240985. Validation accuracy: 93.40%
Epoch 2. Training loss: 0.212. Validation loss: 0.174620. Validation accuracy: 95.28%
Epoch 3. Training loss: 0.166. Validation loss: 0.152502. Validation accuracy: 95.66%
Epoch 4. Training loss: 0.139. Validation loss: 0.129990. Validation accuracy: 96.38%
Epoch 5. Training loss: 0.119. Validation loss: 0.120189. Validation accuracy: 96.52%
Epoch 6. Training loss: 0.105. Validation loss: 0.111894. Validation accuracy: 96.70%
Epoch 7. Training loss: 0.093. Validation loss: 0.108442. Validation accuracy: 96.46%
Epoch 8. Training loss: 0.083. Validation loss: 0.102924. Validation accuracy: 97.00%
Epoch 9. Training loss: 0.075. Validation loss: 0.102051. Validation accuracy: 96.86%
Epoch 10. Training loss: 0.068. Val